# TREC 2018 Precision Medicine

In [1]:
from json2html import *
from IPython.display import HTML
import pandas
import os
import xml.etree.ElementTree
import operator
import pytrec_eval
import eval
import utils
import json

In [2]:
with open('config.json', 'r') as f:
    config = json.load(f)
#HTML(json2html.convert(json = config))

Load TOPICS and GOLD STANDARD from last year (30 topics)

In [3]:
topics_df = eval.get_topics('./topics/topics2017.xml')

In [4]:
topics_df

,topic,disease,gene,demographic
1,1,Liposarcoma,CDK4 Amplification,38-year-old male
2,2,Colon cancer,"KRAS (G13D), BRAF (V600E)",52-year-old male
3,3,Meningioma,"NF2 (K322), AKT1(E17K)",45-year-old female
4,4,Breast cancer,"FGFR1 Amplification, PTEN (Q171)",67-year-old female
5,5,Melanoma,"BRAF (V600E), CDKN2A Deletion",45-year-old female
6,6,Melanoma,NRAS (Q61K),55-year-old male
7,7,Lung cancer,EGFR (L858R),50-year-old female
8,8,Lung cancer,EML4-ALK Fusion transcript,52-year-old male
9,9,Gastrointestinal stromal tumor,KIT Exon 9 (A502_Y503dup),49-year-old female
10,10,Lung adenocarcinoma,KRAS (G12C),61-year-old female


In [5]:
run_df = eval.run('my_run', topics_df)

RUN: my_run
TOPIC: 1 Liposarcoma CDK4 Amplification 38-year-old male
TOPIC: 2 Colon cancer KRAS (G13D), BRAF (V600E) 52-year-old male
TOPIC: 3 Meningioma NF2 (K322), AKT1(E17K) 45-year-old female
TOPIC: 4 Breast cancer FGFR1 Amplification, PTEN (Q171) 67-year-old female
TOPIC: 5 Melanoma BRAF (V600E), CDKN2A Deletion 45-year-old female
TOPIC: 6 Melanoma NRAS (Q61K) 55-year-old male
TOPIC: 7 Lung cancer EGFR (L858R) 50-year-old female
TOPIC: 8 Lung cancer EML4-ALK Fusion transcript 52-year-old male
TOPIC: 9 Gastrointestinal stromal tumor KIT Exon 9 (A502_Y503dup) 49-year-old female
TOPIC: 10 Lung adenocarcinoma KRAS (G12C) 61-year-old female
TOPIC: 11 Gastric cancer PIK3CA (E545K) 54-year-old male
TOPIC: 12 Colon cancer BRAF (V600E) 35-year-old male
TOPIC: 13 Cholangiocarcinoma BRCA2 72-year-old male
TOPIC: 14 Cholangiocarcinoma IDH1 (R132H) 64-year-old male
TOPIC: 15 Cervical cancer STK11 26-year-old female
TOPIC: 16 Pancreatic cancer CDKN2A 54-year-old male
TOPIC: 17 Prostate cancer P

In [6]:
run_df

,TOPIC_NO,Q0,ID,RANK,SCORE,RUN_NAME
0,1,Q0,26336885,1,54.524635,my_run
1,1,Q0,16160477,2,51.825653,my_run
2,1,Q0,9713346,3,50.296030,my_run
3,1,Q0,21910158,4,50.229340,my_run
4,1,Q0,18551309,5,50.125010,my_run
5,1,Q0,11505267,6,49.909904,my_run
6,1,Q0,18820664,7,49.677400,my_run
7,1,Q0,25028469,8,49.263350,my_run
8,1,Q0,21038090,9,49.167870,my_run
9,1,Q0,26464734,10,48.654488,my_run


In [7]:
eval.evaluate(eval.get_qrels('./gold-standard/abstracts.2017.qrels'), run_df)

{'P_10': 0.44, 'Rprec': 0.2626, 'ndcg': 0.4779}